In [1]:
import pandas as pd
import numpy as np

In [2]:
auctions = pd.read_csv('auctions.csv.gzip', compression = 'gzip', low_memory=False)

# Preparo el df y divido ventanas

In [3]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [4]:
w1 = auctions[auctions['date'].dt.day < 21]
w2 = auctions[(auctions['date'].dt.day > 18) & (auctions['date'].dt.day < 22)]
w3 = auctions[(auctions['date'].dt.day > 19) & (auctions['date'].dt.day < 23)]
w4 = auctions[(auctions['date'].dt.day > 20) & (auctions['date'].dt.day < 24)]
w5 = auctions[(auctions['date'].dt.day > 21) & (auctions['date'].dt.day < 25)]
w6 = auctions[(auctions['date'].dt.day > 22) & (auctions['date'].dt.day < 26)]
w7 = auctions[(auctions['date'].dt.day > 23) & (auctions['date'].dt.day < 27)]

In [5]:
windowStart = pd.to_datetime("2019-04-18 00:00:00.000000")
w1['timeFromWindowStart'] = (w1['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-19 00:00:00.000000")
w2['timeFromWindowStart'] = (w2['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-20 00:00:00.000000")
w3['timeFromWindowStart'] = (w3['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-21 00:00:00.000000")
w4['timeFromWindowStart'] = (w4['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-22 00:00:00.000000")
w5['timeFromWindowStart'] = (w5['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w6['timeFromWindowStart'] = (w6['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w7['timeFromWindowStart'] = (w7['date'] - windowStart)/np.timedelta64(1,'s')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [6]:
w1 = w1.sort_values(by=['device_id', 'date'])
w1 = w1.drop_duplicates('device_id', keep = 'first')
w2 = w2.sort_values(by=['device_id', 'date'])
w2 = w2.drop_duplicates('device_id', keep = 'first')
w3 = w3.sort_values(by=['device_id', 'date'])
w3 = w3.drop_duplicates('device_id', keep = 'first')
w4 = w4.sort_values(by=['device_id', 'date'])
w4 = w4.drop_duplicates('device_id', keep = 'first')
w5 = w5.sort_values(by=['device_id', 'date'])
w5 = w5.drop_duplicates('device_id', keep = 'first')
w6 = w6.sort_values(by=['device_id', 'date'])
w6 = w6.drop_duplicates('device_id', keep = 'first')
w7 = w7.sort_values(by=['device_id', 'date'])
w7 = w7.drop_duplicates('device_id', keep = 'first')

In [7]:
w1.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w2.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w3.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w4.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w5.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w6.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w7.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']

In [8]:
w1 = w1.drop(columns = ['date'])
w2 = w2.drop(columns = ['date'])
w3 = w3.drop(columns = ['date'])
w4 = w4.drop(columns = ['date'])
w5 = w5.drop(columns = ['date'])
w6 = w6.drop(columns = ['date'])
w7 = w7.drop(columns = ['date'])

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,82963.592845,75012.201701
33917684,168103949904656,1,3,173044.526198,82963.592845,81005.529793
9402375,186034136943920,1,1,88.851558,82963.592845,75747.944748
13736755,295841792051458,1,1,432.827168,82963.592845,75747.944748
43932908,345999128501141,1,0,164703.549846,82963.592845,75012.201701


## Aplico KNN y RandomForest en la primer ventana y hago las predicciones de la cuarta ventana y despues le voy a agregar esas predicciones como features

In [10]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [11]:
X, y = w1.iloc[:,:-1],w1.iloc[:,-1]

In [12]:
neigh = KNeighborsRegressor(n_neighbors=101)

In [13]:
neigh.fit(X, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=101, p=2,
          weights='uniform')

In [32]:
preds_KNN_w4 = neigh.predict(w4.iloc[:,:-1])

In [15]:
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [16]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [33]:
preds_RF_w4 = rf.predict(w4.iloc[:,:-1])

## Ya me guarde las predicciones de w4 de RF y de KNN y las voy a agregar como features luego.
## Ahora voy a entrenar con w4 y validar con w7 y luego les voy a agregar tanto a w4 como a w7 los features de las predicciones de KNN y de RF

In [34]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [35]:
neigh = KNeighborsRegressor(n_neighbors=101)

In [36]:
neigh.fit(X, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=101, p=2,
          weights='uniform')

In [37]:
preds_KNN_w7 = neigh.predict(w7.iloc[:,:-1])

In [38]:
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [39]:
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [40]:
preds_RF_w7 = rf.predict(w7.iloc[:,:-1])

In [41]:
w4["pred_KNN"] = preds_KNN_w4

In [42]:
w4["pred_RF"] = preds_RF_w4
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7

In [45]:
w4.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [47]:
w4.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,82963.592845,75012.201701
33917684,168103949904656,1,3,173044.526198,82963.592845,81005.529793
9402375,186034136943920,1,1,88.851558,82963.592845,75747.944748
13736755,295841792051458,1,1,432.827168,82963.592845,75747.944748
43932908,345999128501141,1,0,164703.549846,82963.592845,75012.201701


In [48]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,212658.597103,89392.758783,75890.993556
1528620,345999128501141,1,2,336762.048531,89392.758783,87800.256731
20706804,360710529886978,1,1,133087.256609,89392.758783,79290.162968
35163923,365882020742330,1,8,183027.763854,89392.758783,184289.399513
20794330,416301579449694,1,1,98969.446617,89392.758783,79290.162968


## Ahora voy a entrenar w4 y validar con w7 con XBBoost con las nuevas features

In [49]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [50]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [51]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.9, learning_rate = 0.80,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [52]:
xg_reg.fit(X,y)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.8, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
preds = xg_reg.predict(w7.iloc[:,:-1])

In [54]:
rmse = np.sqrt(mean_squared_error(w7.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 7362.910028
